In [199]:
using LightGraphs, SimpleWeightedGraphs
using CPUTime
using DelimitedFiles
using Base
using DataStructures
using Base.Threads


In [241]:
nthreads()

4

In [284]:
# Run this only for diagnostic purposes
numVertex=6
g = SimpleWeightedGraph(numVertex)
Edges = [(1,2,1), (1,3,3), (2,3,1), (2,4,7), (3,5,1), (2,5,3), (4,5,1), (4,6,1), (5,6,6)]; # Manual edges input for initial sanity check, format is (src, dst, weight)
for edge in Edges
    add_edge!(g, edge[1], edge[2], edge[3])
end

In [285]:
function readDataFromFile(filename="in7.txt")
    open(filename, "r") do f 
        n, m = split(readline(f))
        n = parse(Int, n)
        g = SimpleWeightedGraph(n)
        for ln in eachline(f)
#             a,b,c = split(ln)
            a,b,c = readdlm(IOBuffer(ln), Int)
            if(a==0)
                a=n
            end
            if(b==0)
                b=n
            end
            
            if(!(has_edge(g,a,b)) || get_weight!(g, a,b) > c)
                if(c ==0)
                    add_edge!(g, a, b, 0.01)
                else
                    add_edge!(g, a,b,c)
                end
                
            end
        end
        return g
    end
end
# for in_n_10e5_m_5e6.in, generate the file from graph_generator.ipynb
# g = readDataFromFile("in_n_10e5_m_5e6.in")
g = readDataFromFile("in7.txt")

{10000, 100000} undirected simple Int64 graph with Float64 weights

In [298]:
connected_vs = IntDisjointSets(nv(g))

IntDisjointSets([1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  9991, 9992, 9993, 9994, 9995, 9996, 9997, 9998, 9999, 10000], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 10000)

In [299]:
joined_nodes = Dict{Int, Vector{Int}}()
for j in 1:nv(g)
    i = find_root(connected_vs,j)
    if(haskey(joined_nodes,i))
        push!(joined_nodes[i],j)
    else
        joined_nodes[i] = [j]
    end
end

In [300]:
mst = Vector{edgetype(g)}()
sizehint!(mst, nv(g) - 1)
res =0

0

In [301]:
MAX_INT=2000000000;
minCost = fill(MAX_INT, nv(g)+1);
minNodeTgt = Vector{Int}(1:nv(g)+1);
minNodeSrc = Vector{Int}(1:nv(g)+1);

In [302]:
function initMinCostArray()
    for i = 1:nv(g)
        minCost[i] = MAX_INT
        minNodeTgt[i] = i
        minNodeSrc[i] = i
    end
end

initMinCostArray (generic function with 1 method)

In [303]:
function findMinCostVertex()
    for i in keys(joined_nodes)
        # println("Accessing set ",i, " with sources ", joined_nodes[i])
        for src in joined_nodes[i]
            for dst in neighbors(g, src)
                weight = floor(Int,get_weight(g,src,dst))
                # println(src," -> ", dst, "=", weight, " ",  !in_same_set(connected_vs, src, dst) )
                root_src = find_root(connected_vs, src)
                root_dst = find_root(connected_vs, dst)
                if root_src != root_dst
                    if(minCost[root_src] > weight )
                        minCost[root_src] = weight
                        minNodeTgt[root_src] = dst
                        minNodeSrc[root_src] = src
                    end
                end
            end
            
        end
    end
end

findMinCostVertex (generic function with 1 method)

In [304]:
function contractVertex()
    for i in 1:nv(g)
        if(minCost[i]!= MAX_INT && !in_same_set(connected_vs, minNodeSrc[i], minNodeTgt[i]))
            # Connect the vertices, add mst to answer
            set1 = find_root(connected_vs, minNodeSrc[i])
            set2 = find_root(connected_vs, minNodeTgt[i])
            union!(connected_vs, minNodeSrc[i], minNodeTgt[i])
            global res+=minCost[i]
            push!(mst, SimpleWeightedEdge(minNodeSrc[i], minNodeTgt[i], Float64(minCost[i])))
            # Merge Vertices that has been connected together
            merge_target = find_root(connected_vs, minNodeSrc[i])
#             println("Set ",set1,"and set ",set2," will be joined to", merge_target)
            if merge_target!=set1
                for j in joined_nodes[set1]
                    push!(joined_nodes[merge_target],j)
                end
                delete!(joined_nodes,set1)
            end
            if merge_target!=set2
                for j in joined_nodes[set2]
                    push!(joined_nodes[merge_target],j)
                end
                delete!(joined_nodes,set2)
            end
        end
    end
end

contractVertex (generic function with 1 method)

In [305]:
function contractVertex()
    for i in 1:nv(g)
        if(minCost[i]!= MAX_INT && !in_same_set(connected_vs, minNodeSrc[i], minNodeTgt[i]))
            # Connect the vertices, add mst to answer
            set1 = find_root(connected_vs, minNodeSrc[i])
            set2 = find_root(connected_vs, minNodeTgt[i])
            union!(connected_vs, minNodeSrc[i], minNodeTgt[i])
            global res+=minCost[i]
            push!(mst, SimpleWeightedEdge(minNodeSrc[i], minNodeTgt[i], Float64(minCost[i])))
            # println(minNodeSrc[i]," ", minNodeTgt[i]," ", minCost[i])
            # Merge Vertices that has been connected together
            merge_target = find_root(connected_vs, minNodeSrc[i])
#             println("Set ",set1,"and set ",set2," will be joined to", merge_target)
            if merge_target!=set1
                for j in joined_nodes[set1]
                    push!(joined_nodes[merge_target],j)
                end
                delete!(joined_nodes,set1)
            end
            if merge_target!=set2
                for j in joined_nodes[set2]
                    push!(joined_nodes[merge_target],j)
                end
                delete!(joined_nodes,set2)
            end
        end
    end
end

contractVertex (generic function with 1 method)

In [306]:
function boruvka_MST(maxItr = convert(Int64, round(log2(nv(g))+1, digits=0)))
    i=1
    println("Max iteration: ", maxItr)
    while(i<maxItr && length(mst)< nv(g)-1)
        i+=1
#         println(i, " ", length(mst))
        initMinCostArray()
        findMinCostVertex()
        contractVertex()
#         println(res)
#         println(length(mst))
    end
end

boruvka_MST (generic function with 2 methods)

In [307]:
boruvka_MST()

Max iteration: 14
2 0
4426966
3 7492
5739948
4 9478
6018249
5 9919
6056833
6 9986
6063489
7 9998
6064035


In [308]:
res

6064035

In [309]:
length(mst)

9999